In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#!pip install wandb

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix
import numpy as np
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback

In [4]:
wandb.login()

In [5]:
wandb.init(project="Tabular_Data_Kaggle_Competition")

In [6]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')

In [7]:
X = df_train.drop(["id", "f_27", "target"], axis=1)
y = df_train.target.values

In [8]:
cor_train = df_train.corr()
cor_train.target.sort_values(ascending = False).head(10)

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

In [10]:
X.shape[1]

In [15]:
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 15,
  "batch_size": 32
}

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(30,)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

In [18]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=wandb.config['learning_rate']),
             loss='binary_crossentropy',
              metrics = ['acc']
             )

model.summary()

In [19]:
inputs_train = np.array(X_train)
inputs_val = np.array(X_val)

In [20]:
history = model.fit(inputs_train, y_train, 
                    validation_data=(inputs_val, y_val), 
                    epochs=wandb.config['epochs'],
                    batch_size=wandb.config['batch_size'],
                    callbacks=[WandbCallback()]
                   )

In [22]:
wandb.finish()

In [ ]:
model.save('MyTF_Model.h5')

In [ ]:
#MyModel = load_model('MyTF_Model.h5')
#MyModel.summary() 

In [ ]:
X_test = df_test.drop(["id", "f_27"], axis=1)

In [ ]:
preds = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:
sub = pd.DataFrame(np.column_stack((list(df_test['id']), preds)), columns=["id", "target"])

In [ ]:
sub.to_csv("submission.csv", index=False)